# $ \text{Video Killed The Radio Star}$ $\color{red}{...Diffusion}$

Notebook by David Marx ([@DigThatData](https://twitter.com/digthatdata))

Shared under MIT license


# $\text{FAQ}$

**What is this?**

Point this notebook at a youtube url and it'll make a music video for you.

**How does this animation technique work?**

For each text prompt you provide, the notebook will...

1. Generate an image based on that text prompt (using stable diffusion)
2. Use the generated image as the `init_image` to recombine with the text prompt to generate variations similar to the first image. This produces a sequence of extremely similar images based on the original text prompt
3. Images are then intelligently reordered to find the smoothest animation sequence of those frames
3. This image sequence is then repeated to pad out the animation duration as needed

The technique demonstrated in this notebook was inspired by a [video](https://www.youtube.com/watch?v=WJaxFbdjm8c) created by Ben Gillin.

**How are lyrics transcribed?**

This notebook uses openai's recently released 'whisper' model for performing automatic speech recognition. 
OpenAI was kind enough to offer several different sizes of this model which each have their own pros and cons. 
This notebook uses the largest whisper model for transcribing the actual lyrics. Additionally, we use the 
smallest model for performing the lyric segmentation. Neither of these models is perfect, but the results 
so far seem pretty decent.

The first draft of this notebook relied on subtitles from youtube videos to determine timing, which was
then aligned with user-provided lyrics. Youtube's automated captions are powerful and I'll update the
notebook shortly to leverage those again, but for the time being we're just using whisper for everything
and not referencing user-provided captions at all.

**Something didn't work quite right in the transcription process. How do fix the timing or the actual lyrics?**

The notebook is divided into several steps. Between each step, a "storyboard" file is updated. If you want to
make modifications, you can edit this file directly and those edits should be reflected when you next load the
file. Depending on what you changed and what step you run next, your changes may be ignored or even overwritten.
Still playing with different solutions here.

**Can I provide my own images to 'bring to life' and associate with certain lyrics/sequences?**

Yes, you can! As described above: you just need to modify the storyboard. Will describe this functionality in
greater detail after the implementation stabilizes a bit more.

**This gave me an idea and I'd like to use just a part of your process here. What's the best way to reuse just some of the machinery you've developed here?**

Most of the functionality in this notebook has been offloaded to library I published to pypi called `vktrs`. I strongly encourage you to import anything you need 
from there rather than cutting and pasting function into a notebook. Similarly, if you have ideas for improvements, please don't hesitate to submit a PR!

**How can I support your work or work like it?**

This notebook was made possible thanks to ongoing support from [stability.ai](https://stability.ai/). The best way to support my work is to share it with your friends, [report bugs](https://github.com/dmarx/video-killed-the-radio-star/issues/new), [suggest features](https://github.com/dmarx/video-killed-the-radio-star/discussions) or to donate to open source non-profits :) 

## $0.$ Setup

In [ ]:
# @title # 📊 Check GPU Status

import pandas as pd
import subprocess

def gpu_info():
    outv = subprocess.run([
        'nvidia-smi',
            # these lines concatenate into a single query string
            '--query-gpu='
            'timestamp,'
            'name,'
            'utilization.gpu,'
            'utilization.memory,'
            'memory.used,'
            'memory.free,'
            ,
        '--format=csv'
        ],
        stdout=subprocess.PIPE).stdout.decode('utf-8')

    header, rec = outv.split('\n')[:-1]
    return pd.DataFrame({' '.join(k.strip().split('.')).capitalize():v for k,v in zip(header.split(','), rec.split(','))}, index=[0]).T

gpu_info()

In [ ]:
#%%capture

# @title # 🛠️ Setup: dependencies, imports, definitions

# Install dependencies

try: 
    import google.colab
    local=False
except:
    local=True

# TODO: pin versions
    
# local only additional dependencies
if local:
    %pip install pandas torch pillow beautifulsoup4 scipy toolz numpy lxml librosa scikit-learn

# dependencies for both colab and local
%pip install yt-dlp python-tsp stability-sdk[anim_ui] diffusers transformers ftfy accelerate omegaconf
%pip install openai-whisper panel huggingface_hub ipywidgets safetensors keyframed demucs parse


#####################################################

#  Definitions and imports

import copy
import datetime as dt
import gc
import io
from itertools import chain, cycle
import json
import os
from pathlib import Path
import random
import re
import shutil
import string
import subprocess
from subprocess import Popen, PIPE
import time
import warnings

from bokeh.models.widgets.tables import (
    NumberFormatter, 
    BooleanFormatter,
    CheckboxEditor,
)
from diffusers import (
    StableDiffusionImg2ImgPipeline,
    StableDiffusionPipeline,
)
from IPython.display import display
import keyframed
import keyframed as kf # TODO...
import keyframed.dsl
import librosa
import matplotlib.pyplot as plt
import numpy as np
from omegaconf import OmegaConf, DictConfig
import pandas as pd
import panel as pn
import parse
import PIL
from PIL import Image
from PIL import Image, ImageDraw, ImageFont
from python_tsp.exact import solve_tsp_dynamic_programming
from safetensors.numpy import save_file as save_safetensors
from safetensors.numpy import load_file as load_safetensors
import scipy
from scipy.spatial.distance import pdist, squareform
import sklearn.cluster
import textwrap
from tqdm.autonotebook import tqdm
import torch
from torch import autocast
import whisper

from stability_sdk.api import Context
from stability_sdk.animation import AnimationArgs, Animator

from stability_sdk.animation import (
    AnimationArgs,
    Animator,
    AnimationSettings,
    BasicSettings,
    CoherenceSettings,
    ColorSettings,
    DepthSettings,
    InpaintingSettings,
    Rendering3dSettings,
    CameraSettings,
    VideoInputSettings,
    VideoOutputSettings,
)

try: 
    import google.colab
    local=False
except:
    local=True

    
def sanitize_folder_name(fp):
    outv = ''
    whitelist = string.ascii_letters + string.digits + '-_'
    for token in str(fp):
        if token not in whitelist:
            token = '-'
        outv += token
    return outv

# to do: is there a way to check if this is in the env already?
#pn.extension('tabulator')


def establish_workspace(
    use_stability_api,
    mount_gdrive,
    application_name="VideoKilledTheRadioStar",
    active_project=None,
):
    """
    This function constructs a local file called `config.yaml` that maintains state that will be used elsewhere.
    It mostly sets the names of project folders and a handful of settings. The reason for doing things this way
    is to facilitate "resume" functionality and creating new projects without overwriting previously created assets.

    By convention, when loaded the config.yaml is referred to as the `workspace` object.
    
    Most project-specific content will be located in a project-specific config -- `storyboard.yaml` -- which should be
    located in the folder path given by `workspace.project_root`. By convention, when loaded this is referred to as the
    `storyboard` object. 

    If everything is set up correctly, you should be able to load the currently configured workspace and storyboard via:

        workspace, storyboard = load_storyboard()
    """
    # yeah... so... this shouldn't be necessary....
    import os

    # infer if we're on colab or not, since this impacts gdrive mounting
    try: 
        import google.colab
        local=False
    except:
        local=True

    if local:
        mount_gdrive=False

    # Infer directory locations
    os.environ['XDG_CACHE_HOME'] = os.environ.get(
        'XDG_CACHE_HOME',
        str(Path('~/.cache').expanduser())
    )
    if mount_gdrive:
        from google.colab import drive
        drive.mount('/content/drive')
        Path('/content/drive/MyDrive/AI/models/.cache/').mkdir(parents=True, exist_ok=True) 
        os.environ['XDG_CACHE_HOME']='/content/drive/MyDrive/AI/models/.cache'

    model_dir_str=str(Path(os.environ['XDG_CACHE_HOME']))
    proj_root_str = '${active_project}'
    application_root = str(Path('.').absolute())
    if mount_gdrive:
        application_root = '/content/drive/MyDrive/AI/VideoKilledTheRadioStar'


    # Build config file that defines the "workspace" abstraction
    workspace = OmegaConf.create({
        'active_project': active_project if active_project else str(time.time()),
        'application_root':application_root,
        'project_root':"${application_root}/${active_project}",
        'shared_assets_root':"${application_root}/shared_assets",
        'gdrive_mounted':mount_gdrive,
        'use_stability_api':use_stability_api,
        'model_dir':model_dir_str,
        'output_dir':'${project_root}/frames'
    })

    Path(workspace.project_root).mkdir(parents=True, exist_ok=True)
    Path(workspace.model_dir).mkdir(parents=True, exist_ok=True)
    Path(workspace.output_dir).mkdir(parents=True, exist_ok=True)

    ###################

    # Assign tracking locations for A/V assets and generally useful outputs
    
    assets_dir = Path(workspace.shared_assets_root)
    assets_dir.mkdir(parents=True, exist_ok=True)

    # TODO: yaml -> jsonl ?
    video_assets_meta_fname = assets_dir / 'video_assets_meta.yaml'
    if not video_assets_meta_fname.exists():
        video_assets_meta = OmegaConf.create()
        video_assets_meta.videos = []
        with video_assets_meta_fname.open('w') as fp:
            OmegaConf.save(config=video_assets_meta, f=fp.name)
    else:
        video_assets_meta = OmegaConf.load(video_assets_meta_fname)

    audio_assets_meta_fname = assets_dir / 'audio_assets_meta.yaml'
    if not audio_assets_meta_fname.exists():
        audio_assets_meta = OmegaConf.create()
        audio_assets_meta.content = []
        with audio_assets_meta_fname.open('w') as fp:
            OmegaConf.save(config=audio_assets_meta, f=fp.name)
    else:
        audio_assets_meta = OmegaConf.load(audio_assets_meta_fname)

    ###################

    # Request user provide credentials as needed

    # if use_stability_api:
    #     import os, getpass
    #     if not os.environ.get('STABILITY_KEY'):
    #         os.environ['STABILITY_KEY'] = getpass.getpass('Enter your Stability API Key, then press enter to continue')
    # else:
    #     # TODO: check for HF token in environment
    #     if not local:
    #         from google.colab import output
    #         output.enable_custom_widget_manager()
            
    #     from huggingface_hub import notebook_login
    #     notebook_login()
    
    ###################

    with open('config.yaml','w') as fp:
        OmegaConf.save(config=workspace, f=fp.name)

    return workspace

########################

# wrap some of the loading logic for portability

def save_storyboard(storyboard):
    root = Path(load_workspace().project_root)
    storyboard_fname = root / 'storyboard.yaml'
    with open(storyboard_fname) as fp:
        OmegaConf.save(config=storyboard, f=fp.name)

def load_workspace():
    return OmegaConf.load('config.yaml')
        
def load_storyboard():
    workspace = load_workspace()
    root = Path(workspace.project_root)
    storyboard_fname = root / 'storyboard.yaml'
    storyboard = OmegaConf.load(storyboard_fname)
    return workspace, storyboard

def load_audio_meta(workspace, storyboard):
    assets_dir = Path(workspace.shared_assets_root)
    audio_assets_meta_fname = assets_dir / 'audio_assets_meta.yaml'
    audio_assets_meta = OmegaConf.load(audio_assets_meta_fname)
    audio_meta=dict()
    for idx, rec in enumerate(audio_assets_meta.content):
        if rec.audio_fpath == storyboard.params.audio_fpath:
            audio_meta = rec
            break
    return audio_meta



#######################

# EXTRA SEGMENTATION STUFF


def calculate_interword_gaps(segment):
    end_prev = -1
    gaps = []
    for word in segment['words']:
        if end_prev < 0:
            end_prev = word['end']
            continue 
        gap = word['start'] - end_prev
        gaps.append(gap)
        end_prev = word['end']
    return gaps 

def trivial_subsegmentation(segment, threshold=0, gaps=None):
    """
    split on gaps in detected vocal activity. 
    Contiguity = gap between adjacent tokens is less than the input threshold.
    """
    if gaps is None:
        gaps = calculate_interword_gaps(seg)
    out_segments = []
    this_segment = [seg['words'][0]]
    for word, preceding_pause in zip(seg['words'][1:], gaps):
        if preceding_pause <= threshold:
            this_segment.append(word)
        else:
            out_segments.append(this_segment)
            this_segment = [word]
    out_segments.append(this_segment)

    outv = [dict(
        start=seg[0]['start'],
        end=seg[-1]['end'],
        text=''.join([w['word'] for w in seg]).strip(),
    ) for seg in out_segments]

    return outv

##############################################################

# audio processing


def analyze_audio_structure(
    audio_fpath,
    BINS_PER_OCTAVE = 12 * 3, # should be a multiple of twelve: https://github.com/MTG/essentia/blob/master/src/examples/python/tutorial_spectral_constantq-nsg.ipynb
    N_OCTAVES = 7,
):
    """
    via librosa docs
    https://librosa.org/doc/latest/auto_examples/plot_segmentation.html#sphx-glr-auto-examples-plot-segmentation-py
    cites: McFee and Ellis, 2014 - https://brianmcfee.net/papers/ismir2014_spectral.pdf
    """
    y, sr = librosa.load(audio_fpath)

    C = librosa.amplitude_to_db(np.abs(librosa.cqt(y=y, sr=sr,
                                            bins_per_octave=BINS_PER_OCTAVE,
                                            n_bins=N_OCTAVES * BINS_PER_OCTAVE)),
                                ref=np.max)

    # reduce dimensionality via beat-synchronization
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)
    Csync = librosa.util.sync(C, beats, aggregate=np.median)
    
    # I have concerns about this frame fixing operation
    beat_times = librosa.frames_to_time(librosa.util.fix_frames(beats, x_min=0), sr=sr)

    # width=3 prevents links within the same bar 
    # mode=’affinity’ here implements S_rep (after Eq. 8)
    R = librosa.segment.recurrence_matrix(Csync, width=3, mode='affinity', sym=True)
    # Enhance diagonals with a median filter (Equation 2)
    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, 7))
    # build the sequence matrix (S_loc) using mfcc-similarity
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    Msync = librosa.util.sync(mfcc, beats)
    path_distance = np.sum(np.diff(Msync, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance / sigma)
    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)
    # compute the balanced combination
    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)
    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)
    A = mu * Rf + (1 - mu) * R_path

    # compute normalized laplacian and its spectrum
    L = scipy.sparse.csgraph.laplacian(A, normed=True)
    evals, evecs = scipy.linalg.eigh(L)
    # clean this up with a median filter. can help smooth over discontinuities
    evecs = scipy.ndimage.median_filter(evecs, size=(9, 1))
    return dict(
        y=y, 
        sr=np.array(sr).astype(np.uint32),
        tempo=tempo,
        beats=beats,
        beat_times=beat_times,
        evecs=evecs,
    )

    
def laplacian_segmentation(
    audio_fpath=None,
    evecs=None,
    n_clusters = 5,
    n_spectral_features = None,
):
    """
    segment audio by clustering a self-similarity matrix.
    via librosa docs
    https://librosa.org/doc/latest/auto_examples/plot_segmentation.html#sphx-glr-auto-examples-plot-segmentation-py
    cites: McFee and Ellis, 2014 - https://brianmcfee.net/papers/ismir2014_spectral.pdf
    """
    if evecs is None:
        if audio_fpath is None:
            raise Exception("One of `audio_fpath` or `evecs` must be provided")
        features = analyze_audio_structure(audio_fpath)
        evecs = features['evecs']
        
    if n_clusters < 2:
        seg_ids = np.zeros(evecs.shape[0], dtype=int)
        return seg_ids
    
    if n_spectral_features is None:
        n_spectral_features = n_clusters

    # cumulative normalization is needed for symmetric normalize laplacian eigenvectors
    Cnorm = np.cumsum(evecs**2, axis=1)**0.5
    k = n_spectral_features
    X = evecs[:, :k] / Cnorm[:, k-1:k]


    # use these k components to cluster beats into segments
    KM = sklearn.cluster.KMeans(n_clusters=n_clusters, n_init="auto")
    seg_ids = KM.fit_predict(X)

    return seg_ids #, beat_times, tempo


# for video duration
def get_audio_duration_seconds(audio_fpath):
    outv = subprocess.run([
        'ffprobe'
        ,'-i',audio_fpath
        ,'-show_entries', 'format=duration'
        ,'-v','quiet'
        ,'-of','csv=p=0'
        ],
        stdout=subprocess.PIPE
        ).stdout.decode('utf-8')
    return float(outv.strip())


##########################################

# animation stuff

# TODO: update this stuff to reflect updates to API/sdk
def get_image_for_prompt_sai(prompt, max_retries=5, **kargs):
    stability_api = client.StabilityInference(
        key=os.environ['STABILITY_KEY'], 
        verbose=False,
    )

    # auto-retry if mitigation triggered
    while max_retries:
        try:
            answers = stability_api.generate(prompt=prompt, **kargs)
            response = process_response(answers)
            for img in response:
                yield img
            break

        # TODO: better regen handling
        except RuntimeError:
            print("runtime error")
            max_retries -= 1
            warnings.warn(f"mitigation triggered, retries remaining: {max_retries}")

def process_response(answers):
    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
                raise RuntimeError
            if artifact.type == generation.ARTIFACT_IMAGE:
                img = Image.open(io.BytesIO(artifact.binary))
                yield img


########################################

# misc utils

def rand_str(n_char=5):
    return ''.join(random.choice(string.ascii_lowercase) for i in range(n_char))

def save_frame(
    img: Image,
    idx:int=0,
    root_path=Path('./frames'),
    name=None,
):
    root_path.mkdir(parents=True, exist_ok=True)
    if name is None:
        name = rand_str()
    outpath = root_path / f"{idx}-{name}.png"
    img.save(outpath)
    return str(outpath)

def get_image_sequence(idx, root, init_first=True):
    root = Path(root)
    images = (root / 'frames' ).glob(f'{idx}-*.png')
    images = [str(fp) for fp in images]
    if init_first:
        init_image = None
        images2 = []
        for i, fp in enumerate(images):
            if 'anchor' in fp:
                init_image = fp
            else:
                images2.append(fp)
        if not init_image:
            try:
                init_image, images2 = images2[0], images2[1:]
                images = [init_image] + images2
            except IndexError:
                images = images2
    return images

def archive_images(idx, root, archive_root = None):
    root = Path(root)
    if archive_root is None:
        archive_root = root / 'archive'
    archive_root = Path(archive_root)
    archive_root.mkdir(parents=True, exist_ok=True)
    old_images = get_image_sequence(idx, root=root)
    if not old_images:
        return
    print(f"moving {len(old_images)} old images for scene {idx} to {archive_root}")
    for old_fp in old_images:
        old_fp = Path(old_fp)
        im_name = Path(old_fp.name)
        new_path = archive_root / im_name
        if new_path.exists():
            im_name = f"{im_name.stem}-{time.time()}{im_name.suffix}"
            new_path = archive_root / im_name
        old_fp.rename(new_path)


############################

# video compilation stuff

# TODO: Sorting algorithm that can tolerate more than 15-ish frames (GPU?)
def tsp_sort(frames):
    frames_m = np.array([np.array(f).ravel() for f in frames])
    dmat = pdist(frames_m, metric='cosine')
    dmat = squareform(dmat)
    permutation, _ = solve_tsp_dynamic_programming(dmat)
    return permutation

def add_caption2image(
      image, 
      caption, 
      text_font='LiberationSans-Regular.ttf', 
      font_size=20,
      fill_color=(255, 255, 255),
      stroke_color=(0, 0, 0), #stroke_fill
      stroke_width=2,
      align='center',
      ):
    # via https://stackoverflow.com/a/59104505/819544
    wrapper = textwrap.TextWrapper(width=50) 
    word_list = wrapper.wrap(text=caption) 
    caption_new = ''
    for ii in word_list[:-1]:
        caption_new = caption_new + ii + '\n'
    caption_new += word_list[-1]

    draw = ImageDraw.Draw(image)

    # Download the Font and Replace the font with the font file. 
    font = ImageFont.truetype(text_font, size=font_size)
    w,h = draw.textsize(caption_new, font=font, stroke_width=stroke_width)
    W,H = image.size
    x,y = 0.5*(W-w),0.90*H-h
    draw.text(
        (x,y), 
        caption_new,
        font=font,
        fill=fill_color, 
        stroke_fill=stroke_color,
        stroke_width=stroke_width,
        align=align,
    )

    return image

##########################################################

# audioreactivity stuff


def full_width_plot():
    ax = plt.gca()
    ax.figure.set_figwidth(20)
    plt.show()

def display_signal(y, sr, show_spec=True, title=None):
    
#     if show_spec:
#         frame_time = librosa.samples_to_time(np.arange(len(normalized_signal)), sr=sr)
#     else:
#         frame_time = librosa.frames_to_time(np.arange(len(normalized_signal)), sr=sr)
    
    if show_spec:
        librosa.display.waveshow(y, sr=sr)
        if title:
            plt.title(title)
        full_width_plot()
        
        try:
            times = librosa.times_like(y, sr=sr)
            M = librosa.feature.melspectrogram(y=y, sr=sr)
            librosa.display.specshow(librosa.power_to_db(M, ref=np.max),
                             y_axis='mel', x_axis='time')
            full_width_plot()

        except:
            pass
    else:
        frame_time = librosa.frames_to_time(np.arange(len(y)), sr=sr)
        plt.plot(frame_time, y)
        if title:
            plt.title(title)
        full_width_plot()


# https://github.com/pytti-tools/pytti-core/blob/9e8568365cfdc123d2d2fbc20d676ca0f8715341/src/pytti/AudioParse.py#L95
from scipy.signal import butter, sosfilt, sosfreqz

def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='bandpass', output='sos')
    return sos

def butter_bandpass_filter(y, sr, lowcut, highcut, order=10):
    sos = butter_bandpass(lowcut, highcut, sr, order=order)
    y = sosfilt(sos, y)
    return y

########################################################################


def show_storyboard(storyboard=None):
    if storyboard is None:
        workspace, storyboard = load_storyboard()
    reactive_signal_map = {}
    if storyboard.get('audioreactive'):
        reactive_signal_map = storyboard.audioreactive.get('reactive_signal_map')
        
    for idx, rec in enumerate(storyboard.prompt_starts):
        report = f"scene: {idx}\t start: {rec['start']:.2f}" 
        if rec.get('duration_'):
            report += f"\t duration: {rec.get('duration_'):.2f}"
        report += f"\nspoken text: {rec.get('text')}\n"
        
        # TODO: wrap prompt construction logic in a function (better yet use omegaconf substitution variables)
        
        #'_theme':'theme', 'structural_segmentation_label':
        if rec.get('_theme'):
            report += f"theme prompt: {rec['_theme']}\n"
        #f"image prompt: {rec['_prompt']}\n"
        prompt = rec.get('prompt')
        #if not prompt:
        #    prompt = ...
        if prompt:
            report += f"image prompt: {rec['_prompt']}\n"
        
        if rec.get('animation_mode'):
            report += f"animation mode: {rec['animation_mode']}"
        print(report)
        im_path = rec.get('frame0_fpath')
        if im_path and Path(im_path).exists():
            display(Image.open(rec['frame0_fpath']))

        if reactive_signal_map:
            n = rec['frames']
            if n <1:
                continue
            for signal_name in reactive_signal_map.keys():
                if signal_name in rec:
                    curve = kf.dsl.curve_from_cn_string(rec[signal_name])
                    xs = [i for i in range(n)]
                    ys = [curve[i] for i in xs]
                    plt.plot(xs, ys, label=signal_name)
                plt.title(f"scene {idx}")
                plt.xlabel("frame index within scene")
                plt.legend()
            plt.show()

#########################################

def get_path_to_stems():
    workspace, storyboard = load_storyboard()
    assets_root = Path(workspace.application_root) / 'shared_assets'
    #stems_path = root / "stems"
    stems_path = assets_root / "stems"
    stems_outpath = stems_path / 'htdemucs_ft' / Path(storyboard.params.audio_fpath).stem
    return stems_outpath

def ensure_stems_separated():
    stems_outpath = get_path_to_stems()
    stems_path = str(stems_outpath.parent.parent)
    if not stems_outpath.exists():    
        !demucs -n htdemucs_ft -o "{stems_path}" "{storyboard.params.audio_fpath}"

def get_stem(instrument_name):
    ensure_stems_separated()
    stems_outpath = get_path_to_stems()
    stem_fpaths  = list(stems_outpath.glob('*.wav'))

    for stem_fpath in stem_fpaths:
        if instrument_name in str(stem_fpath):
            y, sr = librosa.load(stem_fpath)
            return y, sr
    raise ValueError(
        f"Unable to locate stem for instrument: {instrument_name}\n"
        f"in folder: {stems_outpath}"
    )


In [ ]:
# @title # 📋 Attach Storyboard (create or resume project)

# @markdown Depending on your settings and environment, running this cell may prompt you to enter one or more API Keys below. 
# @markdown Don't forget to press "enter" after providing a requested key.

# @markdown ---

use_stability_api = False # @param {type:'boolean'}
mount_gdrive = True # @param {type:'boolean'}
resume=True # @param {type:'boolean'}

# TODO: add support for whisper API

# @markdown ---


# @markdown To create a new project, enter a unique project name.
# @markdown If you leave `project_name` blank, the current unix timestamp will be used
# @markdown  (seconds since 1970-01-01 00:00).

# @markdown If you use the name of an existing project, the workspace will switch to that project.

# @markdown Non-alphanumeric characters (excluding '-' and '_') will be replaced with hyphens.

project_name = '' # @param {type:'string'}

##########################

try: 
    import google.colab
    local=False
except:
    local=True

if local:
    mount_gdrive=False


##################################################################

resuming = False
if resume:
    try:
        workspace, storyboard = load_storyboard()
        print("loading storyboard")
        resuming=True
    except:
        resuming = False

if not resuming:
    if not project_name:
        project_name = str(time.time())
    project_name = sanitize_folder_name(project_name)
    
    print("creating workspace")

    workspace = establish_workspace(
        use_stability_api=use_stability_api,
        mount_gdrive=mount_gdrive,
        application_name="VideoKilledTheRadioStar",
        active_project=project_name,
    )

    print("creating new storyboard")
    storyboard = OmegaConf.create()
    storyboard.params = {}

if workspace.use_stability_api:
    import os, getpass
    if not os.environ.get('STABILITY_KEY'):
        os.environ['STABILITY_KEY'] = getpass.getpass('Enter your Stability API Key, then press enter to continue')
else:
    # TODO: check for HF token in environment
    if not local:
        from google.colab import output
        output.enable_custom_widget_manager()
        
    from huggingface_hub import notebook_login
    notebook_login()

##################################################################

root = Path(workspace.project_root)

assets_dir = Path(workspace.shared_assets_root)

video_assets_meta_fname = assets_dir / 'video_assets_meta.yaml'
audio_assets_meta_fname = assets_dir / 'audio_assets_meta.yaml'

video_assets_meta = OmegaConf.load( assets_dir/'video_assets_meta.yaml' )
audio_assets_meta = OmegaConf.load( assets_dir/'audio_assets_meta.yaml' )

    

In [ ]:
# @title Set Audio Source

d_ = dict(
    # all the underscore does is make it so each of the following lines can be preceded with a comma
    # otw the first parameter would be offset from the other in the colab form
    _=""

    , video_url = 'https://www.youtube.com/watch?v=4XNXLasyAKE' # @param {type:'string'}
    , audio_fpath = '' # @param {type:'string'}
)
d_.pop('_')

# @markdown `video_url` - URL of a youtube video to download as a source for audio and potentially for text transcription as well.

# @markdown `audio_fpath` - Optionally provide an audio file instead of relying on a youtube download. Name it something other than 'audio.mp3', 
# @markdown                 otherwise it might get overwritten accidentally.


storyboard.params = d_

storyboard_fname = root / 'storyboard.yaml'
with open(storyboard_fname,'wb') as fp:
    OmegaConf.save(config=storyboard, f=fp.name)
    

###############################
# Download audio from youtube #
###############################

# this should modify the existing record for the URL rather than creating a new one...
force_redownload=False

video_url = storyboard.params.video_url
download_video=True

if not force_redownload:
    for rec in video_assets_meta.videos:
        if rec.video_url == video_url:
            if rec.get('video_fpath'):
                print("previously downloaded video detected")
                download_video=False
                # populate storyboard with previous processing results
                if rec.get('audio_fpath'):
                    storyboard.params.audio_fpath = rec.get('audio_fpath')
            else:
                download_video=True # should be redundant?
            break
            


if download_video:
    # check if user provided an audio filepath (or we already have one from youtube) before attempting to download
    video_assets_meta_record = {}
    video_assets_meta_record['video_url'] = video_url

    ytdl_prefix = "DOWNLOADED__"
    ytdl_fname = f"{str(assets_dir / ytdl_prefix)}%(title)s.%(ext)s"

    !yt-dlp -o "{ytdl_fname}" {video_url}

    matched_files = assets_dir.glob(ytdl_prefix+"*")
    most_recent_file = max(matched_files, key=os.path.getctime)
    print(f"downloaded: {most_recent_file}")
    ytdl_fname = most_recent_file

    video_assets_meta_record['video_fpath'] = str(ytdl_fname.absolute())
    
    # TODO: right here, we should be adding this to the audio meta
    audio_fpath = ytdl_fname.with_suffix('.m4a')
    input_audio = ytdl_fname
    !ffmpeg -y -i "{input_audio}" -vn -c:a aac "{audio_fpath}"

    storyboard.params.audio_fpath = audio_fpath
    video_assets_meta_record['audio_fpath'] = str(audio_fpath.absolute())
    
    video_assets_meta.videos.append(video_assets_meta_record)


save_storyboard(storyboard)

with open(video_assets_meta_fname, 'wb') as fp:
    OmegaConf.save(config=video_assets_meta, f=fp.name)



In [ ]:
# @markdown # 🔊 💬 Audio Processing

# @markdown * Transcribe and segment speech using whisper

# @markdown If this audio source has been previoussly processed by this notebook, that should be detected processing won't be repeated.

##################################################
# 💬 Transcribe and segment speech using whisper #
##################################################

# TODO: combine this with previous cell, doesn't need to be separate step.

#audio_fpath = Path(storyboard.params.audio_fpath)
audio_fpath = str(storyboard.params.audio_fpath)

separate_stems = True # @param {type:'boolean'}

force_retranscription = False # @param {type:'boolean'}
override_storyboard_transcription = False

whisper_seg = None
#audio_meta={}

# this is silly.
# for audio_meta in audio_assets_meta.content:
#     #if (audio_fpath) and (audio_meta.audio_fpath == audio_fpath):
#     if (audio_fpath) and (audio_meta.audio_fpath == audio_fpath):

if audio_fpath in audio_assets_meta:
    print("previously processed audio detected")
    # TODO: add processed audio structure features/stems as well
    audio_meta = audio_assets_meta[audio_fpath]
else:
    audio_assets_meta[audio_fpath] = {}
    audio_meta = audio_assets_meta[audio_fpath]

if (not force_retranscription) and audio_meta.get('whisper_segmentation') and Path(audio_meta.whisper_segmentation).exists():
    print("Using pre-existing whisper transcription")
    whisper_seg_fpath = Path(audio_meta.whisper_segmentation)
    with whisper_seg_fpath.open() as f:
        timings = json.load(f)
    whisper_seg = timings['segments']


if force_retranscription or (whisper_seg is None):
    print("Transcribing...")
    #audio_meta['audio_fpath'] = storyboard.params.audio_fpath redundant
    # outputs text files as audio.* locally
    !whisper --model large --word_timestamps True -o {str(assets_dir)} "{storyboard.params.audio_fpath}"

    whisper_seg_fpath = Path(storyboard.params.audio_fpath).with_suffix('.json')
    audio_meta['whisper_segmentation'] = str(whisper_seg_fpath)
    audio_meta['duration'] = get_audio_duration_seconds(audio_fpath)
    
    with whisper_seg_fpath.open() as f:
        timings = json.load(f)
    whisper_seg = timings['segments']
    
    audio_assets_meta[audio_fpath] = audio_meta
    with open(audio_assets_meta_fname, 'wb') as fp:
        OmegaConf.save(config=audio_assets_meta, f=fp.name)
    
if not storyboard.get('prompt_starts') or override_storyboard_transcription:
    
    # Ta da!
    #storyboard.prompt_starts = whisper_seg
    storyboard.prompt_starts = [{k:rec[k] for k in ('start','end','text')} for rec in whisper_seg]
    
    storyboard.params['video_duration'] = audio_meta['duration']
    # unsure if below method is reliable.
    #storyboard.params['video_duration'] = storyboard.prompt_starts[-1]['end']


### checkpoint the processing work we've done to this point


# TODO: enforce scene zero starts at t=0 and last scene ends at duration

#storyboard.prompt_starts = prompt_starts
save_storyboard(storyboard)

# Music Structure Analysis

# Music Structure Analysis
# - beat and tempo detection
# - Self-similarity graph

# TODO: detect previously processed audio

# TODO: move stem separation here?

# TODO: no decisions here, combine with other cells

audio_structure_features = analyze_audio_structure(audio_fpath=storyboard.params.audio_fpath)

audio_features_fpath = Path(storyboard.params.audio_fpath).with_suffix('.audio_features.safetensors')
save_safetensors(audio_structure_features, audio_features_fpath)

# TODO: fix inconsistent variable naming
structural_features = audio_structure_features

# TODO: un-"PosixPath" `storyboard.params.audio_fpath`
audio_assets_meta[str(storyboard.params.audio_fpath)]['structural_features'] = audio_features_fpath


with open(audio_assets_meta_fname, 'wb') as fp:
    OmegaConf.save(config=audio_assets_meta, f=fp.name)

if separate_stems:
  ensure_stems_separated()

# TODO: ~PLACEHOLDER~ Give user opportunity to correct the transcription at shared_asset level rather than project


In [ ]:
# @title Subdivide Unusually Long Scenes

# @markdown Break up "outlier" segments into smaller chunks

# TODO: wrap these steps in functions for legibility/portability
# TODO: make this threshold parameterizable via storyboard (and save analysis to storyboard)
# TODO: use beat counts to estimate a smart scene duration

subdivide_long_scenes = False # @param {'type':'boolean'}

#TODO: merge short scenes

# TODO: expose this to colab
threshold_duration = None 

######################################################
# estimate parameters of scene duration distribution #
######################################################

# TODO: use beat onsets/counts
scene_durations = []
scenes_ = []
for idx, rec in enumerate(storyboard.prompt_starts):
    rec=dict(rec)
    if idx > 0:
        # are we maybe doubling up 'start' time stamps? like there are more unique 'end's than 'start's?
        duration = rec['start'] - prev['start']
        prev['duration_'] = duration
        scene_durations.append(duration)
    prev = rec
    scenes_.append(prev)
    
# handle last record
else:
    # here's the bug.
    # TODO: swap out rec['end'] -> rec_prev['start'] here
    rec['duration_'] = rec['end'] - rec['start']
    scenes_.append(rec)


mu = sum(scene_durations)/len(scene_durations)
sigma = np.std(scene_durations)

# 1sd filter to concentrate on mode
scene_durations2 = [s for s in scene_durations if (mu - sigma) < s < (mu+sigma)]
mu2 = sum(scene_durations2)/len(scene_durations2)
sigma2 = np.std(scene_durations2)

###########



# break up "outlier" segments into smaller chunks
# this heuristic could be improved with beat synchronization and onset detection.
# ... also could probably leverage the 'end' time of the scene
# TODO: hierarchical theme structure analysis
# TODO: MSA segmentation for fully instrumental (i.e. arbitrary) audio

if subdivide_long_scenes:

    threshhold = mu2 + sigma
    if threshold_duration is not None:
        threshhold = threshold_duration

    scenes = []
    #for rec in storyboard.prompt_starts:
    for rec in list(scenes_):
        gap_remaining = rec['duration_']
        while gap_remaining > threshhold:
            #step = min(max(mu2-sigma, (np.random.normal() + mu2)*sigma), mu2+sigma)
            step=mu2
            step = float(step)
            # TODO: move duration computation somewhere that it will happen necessarily
            rec['duration_'] = step
            new_rec = copy.deepcopy(rec)
            new_rec['start'] = rec['start'] + step
            # TODO: deal with new value here
            #new_rec['end'] = ??
            new_rec['duration_'] = step
            ### maybe i could add a flag or something to clarify that this was an "inferred" subscene
            #new_rec['parent_scene'] = rec.get('uid') # something like this?
            new_rec['inferred_subscene'] = True # or this?
            
            scenes.append(rec)
            rec = new_rec
            gap_remaining -= step
        scenes.append(rec)

    storyboard.prompt_starts = scenes

# TODO: get last scene "end" from story duration

# TODO: compute duration regardless of extra segmentation

# TODO: add scene indices back to rec's to facilitate editing the text file
    
save_storyboard(storyboard)

In [ ]:
# @title Theme -> Scene Assignment

# @markdown `theme_prompt` - Text that will be appended to the end of each lyric, useful for e.g. applying a consistent aesthetic style.
# @markdown  To provide multiple themes (only one will be used per scene), separate theme prompts with the `|` (pipe) symbol.

# @markdown `infer_thematic_structure` - if False, themes will be rotated sequentially such that no two adjacent frames
# @markdown will use the same theme prompt (if multiple theme prompts were provided). If True, song structure analysis will cluster related scenes into as many groups as there are theme prompts to attempt to associate a visual themes with respective musical themes.

# @markdown The analysis runs quick. If you didn't understand that explanation, just try it both ways and you'll probably get the idea.

theme_prompt = 'Katsuhiro Otomo gundam mecha ' # @param {type:'string'}

infer_thematic_structure = True # @param {type:'boolean'}

storyboard.params.theme_prompt = theme_prompt
themes = [prompt.strip() for prompt in theme_prompt.split('|') if prompt.strip()]

if (len(themes) > 1):
    if infer_thematic_structure:
                
        # audio_assets_meta[storyboard.params.audio_fpath]['structural_features']
        beat_times = audio_structure_features['beat_times']
        evecs = audio_structure_features['evecs']
        segment_labels = laplacian_segmentation(
            evecs=evecs,
            
            # TODO: publish these parameters to the user
            n_clusters=len(themes), # be sure to explain this is an upper bound, stochastic
            n_spectral_features=len(themes),
        )
        
        # beatsynch scene start times

        # TODO: swap out rec['end'] -> rec_prev['start'] here
        for rec in storyboard.prompt_starts:
            beat_indices = np.where((beat_times >= rec['start']) & (beat_times <= rec['end']))[0]
            segments_this_interval = segment_labels[beat_indices]
            if len(segments_this_interval) == 0:
                dominant_label = 0
            else:
                dominant_label = int(np.argmax(np.bincount(segments_this_interval)))
            rec['structural_segmentation_label'] = dominant_label
            rec['_theme'] = themes[dominant_label]
    else:
        for rec in storyboard.prompt_starts:
            rec['_theme'] = themes[idx % len(themes)]
else:
    for rec in storyboard.prompt_starts:
        rec['_theme'] = theme_prompt


save_storyboard(storyboard)
show_storyboard(storyboard)

In [ ]:
# @markdown (Optional) modify theme prompt without impacting structure label assignments

# @markdown Themes will be assigned to the `structural_segmentation` label that maps to their ordering
# @markdown in the theme prompt. To change which theme goes where, simply modify the order in which
# @markdown they appear in your prompt.

theme_prompt = 'rusted industrial machinery | kaiju robot CGI | paperclips! paperclips! |  robotics for beginners | rusted industrial machinery' # @param {type:'string'}

#####################################################

storyboard.params.theme_prompt = theme_prompt
themes = [prompt.strip() for prompt in theme_prompt.split('|') if prompt.strip()]

for rec in storyboard.prompt_starts:
    theme_idx = rec.get('structural_segmentation_label',0)
    rec['_theme'] = themes[theme_idx]

save_storyboard(storyboard)
show_storyboard(storyboard)

In [ ]:
# @title OPTIONAL: Assign Animation parameters

# @markdown Run This cell to reveal a multi-tab UI for specifying `img2img` animation settings. When you are happy with your settings, 
# @markdown run the cell which follows this one to attach the settings you have selected to one or more scenes/themes.

# @markdown To reset values to default, simply re-run this cell.

# @markdown The `img2img` animation mode is currently only supported through the Stability.AI Animation API (i.e. DreamStudio API). 
# @markdown The `img2img` animation parameters are mostly compatible with deforum: if you don't want to animate via the Stability.AI,
# @markdown you can still use this notebook to configure your animation and then port the relevant settings from the `storyboard.yaml`
# @markdown to your tool of choice.

if not workspace.use_stability_api:
    warnings.warn("img2img animation currently only supported if you're using the stability API")
    arg_objs=[]
else:


    # @markdown To get your API key visit https://dreamstudio.ai/account
    STABILITY_HOST = "grpc.stability.ai:443" #@param {type:"string"}

    show_documentation = True # @param {type:'boolean'}


    ###################################

    STABILITY_KEY = os.environ.get('STABILITY_KEY')

    # Connect to Stability API
    context = Context(STABILITY_HOST, STABILITY_KEY)

    # Test the connection
    context.get_user_info()



    ###################

    args_generation = BasicSettings()
    args_animation = AnimationSettings()
    args_camera = CameraSettings()
    args_coherence = CoherenceSettings()
    args_color = ColorSettings()
    args_depth = DepthSettings()
    args_render_3d = Rendering3dSettings()
    args_inpaint = InpaintingSettings()
    args_vid_in = VideoInputSettings()
    args_vid_out = VideoOutputSettings()
    arg_objs = (
        args_generation,
        args_animation,
        args_camera,
        args_coherence,
        args_color,
        args_depth,
        args_render_3d,
        args_inpaint,
        args_vid_in,
        args_vid_out,
    )

    def _show_docs(component):
        cols = []
        for k, v in component.param.objects().items():
            if k == 'name':
                continue
            col = pn.Column(v, v.doc)
            cols.append(col)
        return pn.Column(*cols)

    def build(component):
        if show_documentation:
            component = _show_docs(component)
        return pn.Row(component, width=1000)

pn.extension()

pn.Tabs(*[(a.name[:-5], build(a)) for a in arg_objs])

In [ ]:
# @title Attach settings to scenes

# @markdown If using img2img animation, use this cell to specify which scenes the settings you provided above should apply to.

# @markdown Otherwise, you can ignore the cell above and just do everything here.

# @markdown ---

# @markdown `override_storyboard` - If settings conflict with values already set on the storyboard, the values on the storyboard will take priority.

# @markdown `scene_ids` - Comma separated list of integers specifying scenes to attach to.

# @markdown `theme_ids` - Comma separated list of integers specifying themes the animation parameters should be associated with.

# @markdown NB: both `scene_ids` and `theme_ids` are zero-indexed.


# CURRENT ARGS APPLY TO...

all_scenes = True # @param {'type':'boolean'}

scene_ids = '' # @param {'type':'string'}
theme_ids = '' # @param {'type':'string'}

override_storyboard = True # @param {'type':'boolean'}

# @markdown ---

# @markdown ## Animation Modes. 
# @markdown * `static`: turns off animation, static image for duration of scene.
# @markdown * `variations`: injects a small bit of "life" into the image. Cheap and fast.
# @markdown * `variations tsp`: variations animation with frames reordered for smoother motion. Cheap and fast, but a tad slower.
# @markdown * `img2img`: Fancy deforum-esque animation, only supported for stability api at present. Neither cheap nor fast.
# @markdown * `default`: `img2img` if stability api enabled,  `variations tsp` if not. 

# oh baby `jittered init`
# markdown * `jittered init`: The VKTRS special! Sample a variation and use that as an init image for img2img. Basically built for audioreactivity
# to do --> combine this with a referring expression mask
# make sure animation is configured so we can curve prompts

animation_mode = 'default' # @param ["default", "static", "variations", "variations tsp", "img2img"]

# @markdown ---

# @markdown ## Parameters for `variations` Animation Modes

# TODO: these parameters should get set elsewhere. maybe with animation mode?

# @markdown `n_variations` - How many unique variations to generate for a given text prompt. This determines the frequency of the visual "pulsing" effect

# @markdown `image_consistency` - controls similarity between images generated by the prompt.
# @markdown - 0: ignore the init image
# @markdown - 1: true as possible to the init image

## @markdown `max_video_duration_in_seconds` - Early stopping if you don't want to generate a video the full duration of the provided audio. Default = 5min.


n_variations=8 # @param {type:'integer'}
image_consistency=0.72 # @param {type:"slider", min:0, max:1, step:0.01}  

variations_settings = {
    'n_variations':n_variations,
    'image_consistency':image_consistency,
}


# let's try this:
#   - user specifies animation stuff in cell above
#   - then user comes down here and persists the animation settings either to every scene, or to specific scenes.
#   - scene specification can just be a list of numbers
#   - feels like this should be moved after init image generation. maybe push init image generation up to run right after theme prompts and scene count is finalized?

# TODO: add storyboard -> parsec converter

# TODO: PR to deforum for storyboard.yaml support

# TODO: allow user to constrain attention to specific parameters or sets of parameters to set

def param2json(args_obj):
    args = args_obj.param.serialize_parameters()
    args = json.loads(args)
    args.pop('name')
    return args

def collect_animation_args():
    if 'arg_objs' in locals():
        args_d = {}
        [args_d.update(a.param.values()) for a in arg_objs]
        args=AnimationArgs(**args_d)
    else:
        import warnings
        warnings.warn(
            "Looks like you're animating in img2img mode "
            "without having specified any animation parameters. "
            "Using SDK defaults. "
        )
        args = AnimationArgs()
    return param2json(args)


##########################

scene_ids = [int(v.strip()) for v in scene_ids.split(',') if v]
theme_ids = [int(v.strip()) for v in theme_ids.split(',') if v]

# build list of scenes args will apply to

applicable_scenes = []
if all_scenes:
    applicable_scenes = [idx for idx, _ in enumerate(storyboard.prompt_starts)]
else:
    applicable_scenes += scene_ids
    applicable_scenes += theme_ids

###---------------------------------###

animation_mode = animation_mode.lower()
if animation_mode == 'default':
    animation_mode = 'img2img' if workspace.use_stability_api else 'variations tsp'

###---------------------------------###

# apply args to appropriate scenes

for idx, rec in enumerate(storyboard.prompt_starts):
    if idx not in applicable_scenes:
        continue

    board_args = rec.get('animation_args')
    if board_args:
        board_args = OmegaConf.to_container(board_args) # coerce to dict

    new_args = collect_animation_args()
    if not override_storyboard:
        new_args.update(board_args)
        #variations_settings
    rec['animation_args'] = new_args
    rec.update(variations_settings)

    if override_storyboard or (not rec.get('animation_mode')):
        rec['animation_mode'] = animation_mode
        
    # oooo lookat me being fancy with custom code!
    #if (idx >=23) or idx ==16:
    #    rec['animation_mode'] = 'img2img'

save_storyboard(storyboard)
show_storyboard()

In [ ]:
# @title Math

#################################################
# Math                                          #
#                                               #
#    This block computes how many frames are    #
#    needed for each segment based on the start #
#    times for each prompt                      #
#################################################

# TODO: leverage previous beat detection, onsets, etc. for frame timings
# - TODO: isolated cells for calculating and suggesting parameters (fps, n_variations)

# TODO: experiment with tying instantaneous framerate to a music attribute (i.e. so motion can change speed mid animation)

fps = 30 # @param {type:'integer'}
storyboard.params.fps = fps

ifps = 1/fps

# estimate video end
if not storyboard.params.get('video_duration'):
    storyboard.params['video_duration'] = get_audio_duration_seconds(storyboard.params.audio_fpath)
video_duration = storyboard.params['video_duration']

# dummy prompt for last scene duration
prompt_starts = OmegaConf.to_container(storyboard.prompt_starts)
prompt_starts.append({'start':video_duration})

# TODO: do we still need anim_start? is that used in rendering?
# TODO: add per-frame timings to incorporate beat information
# make sure we respect the duration of the previous phrase
frame_start=0
prompt_starts[0]['anim_start']=frame_start
for i, rec in enumerate(prompt_starts[1:], start=1):
    rec_prev = prompt_starts[i-1]
    k=0
    while (rec_prev['anim_start'] + k*ifps) < rec['start']:
        k+=1
    k-=1
    rec_prev['frames'] = k
    rec_prev['anim_duration'] = k*ifps
    frame_start+=k*ifps
    rec['anim_start']=frame_start

# drop the dummy frame
prompt_starts = prompt_starts[:-1]

# TODO: given a 0 duration prompt, assume its duration is captured in the next prompt 
#        and guesstimate a corrected prompt start time and duration 
#      - or rather.. why are there ever sero duration prompts?


storyboard.prompt_starts = prompt_starts

save_storyboard(storyboard)



In [ ]:
# @markdown # Audioreactivity

# @markdown ### 1. Choose a driving signal

driving_signal_name = "bass stem" #@param ['default','user specified','vocals stem','bass stem','other stem','drum stem']

# TODO: test this
custom_signal_fpath = '' # @param {'type':'string'}

def get_user_specified_signal():
    y, sr = librosa.load(custom_signal_fpath)
    return y, sr

y = structural_features['y']
sr = structural_features['sr']

driving_signals = {
    'default': lambda: (y, sr),
    'user specified': get_user_specified_signal,
    'vocals stem':lambda: get_stem('vocals'),
    'bass stem':lambda: get_stem('bass'),
    'other stem':lambda: get_stem('other'),
    'drum stem':lambda: get_stem('drum'),
}

driving_signal, sr = driving_signals[driving_signal_name]()
display_signal(driving_signal, sr)

### Audio EQ cheat sheet

![Audio EQ cheat sheet](https://i.pinimg.com/736x/4f/28/5e/4f285e3fbc5b6b6ea78638e58b2e3052.jpg)

In [ ]:
# @title 2. Manipulate the signal

import numpy as np
from scipy import signal
from inspect import signature
from functools import partial
from scipy.signal import find_peaks
from sklearn.cluster import KMeans
#sklearn_extra.cluster.KMedoids

# @markdown To apply multiple operations, separate manipulation names with a '|' or just re-run this cell.
# @markdown When you're satisfied, run the next cell to replace the loaded driving signal with the manipulated signal.

# @markdown ### Available Signal Manipulations
# @markdown NB: if a signal's name is followed by an asterisk, this means it modifies the signal's time domain. Only one time domain transformation per sequence is supported. 
# @markdown * `raw` - no op
# @markdown * `rms`* - Root mean squared. Converts raw signal to signal power
# @markdown * `novelty`* - Estimates strength of sound event onsets.
# @markdown * `predominant_pulse`* - Combined estimate of beat timing and strength
# @markdown * `stretch` - Increases gap between high and low amplitude signals. Alias for `pow2`
# @markdown * `smoosh` - Reduces gap between high and low amplitude signals. Alias for `sqrt`
# @markdown * `pow(k)` - Raises signal to the power of k.
# @markdown * `pow2` - Square the signal. Increases gap between high and low amplitude signals
# @markdown * `sqrt` - Square root of signal. Reduces gap between high and low amplitude signals
# @markdown * `normalize` - Transform signal to `[0,1]` range. This will always be the last step, even if you don't specify it.
# @markdown * `smooth(k)` - Smoothes out the waveworm, using a smoothing window of `k`. Bigger `k` = flatter signal
# @markdown * `sustain(k)` - Treats signal as a self-exciting process that decays slowly over a window of `k`. 
# @markdown * `bandpass(low, high)` - Isolate signal to frequencies between `[low, high]`
# @markdown * `threshold(low)` - Zero the signal where amplitude is less than `low`
# @markdown * `clamp(high)` - Clamp the signal such that no values have amplitude greater than `high`
# @markdown * `modulo(k)` - Detects peaks in signal and returns only every Kth-peak
# @markdown * `quantize(k)` - Discretizes the signal into K unique values after clustering amplitudes.





# TODO: specify which transforms are available in spectral space vs. not


# trim/trim(q) -> y[y>quantile(y,.1)]
# that 4part way of parameterizing a wave... hit, sustain, decay,..?



# all operations must either have signature (y, sr) or return a function which does

# def rms(y, sr):
#     return librosa.feature.rms(y=y)

# def novelty(y, sr):
#     return librosa.onset.onset_strength(y, sr)

# def predominant_pulse(y, sr):
#     return librosa.beat.plp(y, sr)

def pow2(y, sr):
    return y**2

def sqrt(y, sr):
    return y**-2


# so... apparently `pow` is a python builtin. whoops. Meh, fuck it.
def _pow(k):
    def pow_(y, sr):
        return y**k
    return pow_

def stretch(k=2):
    return _pow(k)

def smoosh(k=2):
    return _pow(-k)

def normalize(y, sr):
    normalized_signal = np.abs(y).ravel()
    normalized_signal /= max(normalized_signal)
    return normalized_signal
    
######################################
    
def smooth(k=150):
    k=int(k)
    def smooth_(y, sr=None):
        win_smooth = signal.windows.hann(k)
        filtered = signal.convolve(y, win_smooth, mode='same') / sum(win_smooth)
        return filtered
    return smooth_

def sustain(k=500):
    k=int(k)
    def sustain_(y, sr=None):
        win_sustain = signal.windows.hann(2*k)
        win_sustain[:k]=0
        filtered = signal.convolve(y.ravel(), win_sustain, mode='same') / sum(win_sustain)
        return filtered
    return sustain_


# TODO: decay() - sustain with an exponential window

#####################333
    
def bandpass(low: float, high:float):
    return partial(butter_bandpass_filter, lowcut=low, highcut=high)
    
def threshold(low):
    def f(y, sr):
        y[y<low] = 0
        return y
    return f

def clamp(high):
    def f(y, sr):
        y[y>high] = high
        return y
    return f


###############################3


# def peak_detection(y, sr):
#     peaks, _ = find_peaks(y)
#     return peaks

# TODO: support offset, so user could e.g. take either every even or every odd peak. 
def modulo(k=2, offset=0):
    #k=int(k)
    def modulo_(y, sr=None):
        #peaks = peak_detection(y, sr)
        peaks, _ = find_peaks(y)
        #selected_peaks = peaks[::k]  # Select every kth peak
        selected_peaks=[]
        for peak_index, peak in enumerate(peaks.ravel()):
            if (peak_index + offset) % k == 0:
                selected_peaks.append(peak)
        print(selected_peaks)
        selected_peaks = np.array(selected_peaks)
        new_signal = np.zeros_like(y)
        new_signal[selected_peaks] = y[selected_peaks]  # Build a new signal with only the selected peaks
        return new_signal
    return modulo_

#################################

# chatgpt wrote this, needs to be tested. also, i might want to use medoids rather than means



def quantize(k=1):
    k=int(k)
    # why doesn't it respect `k` in the closure scope? Works fine for modulo(). weird.
    #def quantize_(y, sr=None):
    def quantize_(y, sr=None, K=k):
        k=K
        # Remove zero values
        nonzero_values = y[y > 0].reshape(-1, 1)
        
        # If the number of nonzero values is less than k, reduce k
        if nonzero_values.shape[0] < k:
            k = nonzero_values.shape[0]
        
        # Perform k-means clustering
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(nonzero_values)

        # Replace each value with the centroid of its cluster
        print(f"cluster centers: {np.unique(kmeans.cluster_centers_)}")
        quantized_values = kmeans.cluster_centers_[kmeans.labels_].flatten()

        # Create a new signal
        quantized_signal = np.zeros_like(y)
        quantized_signal[y > 0] = quantized_values
        return quantized_signal
    return quantize_

#####################################3

# TODO: add ability for user to do stuff via idiomatic `keyframed` rather than convolving signals

# # TODO: add operations: slice/isolate, mute, shift_y/truncate/drop (subtract some value from amplitude)
simple_signal_operations = {
    'raw': lambda y, sr: y,
    ##########
    'rms': lambda y, sr: librosa.feature.rms(y=y),
    'novelty': librosa.onset.onset_strength,
    'predominant_pulse': librosa.beat.plp,
    'bandpass': bandpass,
    'harmonic': lambda y, sr: librosa.effects.harmonic(y=y),
    'percussive': lambda y, sr: librosa.effects.percussive(y=y),
    ##########
    'pow2': lambda y, sr: y**2,
    'stretch': stretch,
    'sqrt': sqrt,
    'smoosh': smoosh,
    'pow':_pow,
    #################
    'smooth': smooth,
    'sustain': sustain,
    # #########
    'normalize': normalize,
    'abs': lambda y, sr: np.abs(np.abs(y)), 
    'threshold': threshold,
    'clamp': clamp,
    'modulo': modulo,
    'quantize':quantize,
}

def prepare_operation(op_str, operations=simple_signal_operations):
    op_str = op_str.replace(' ','') # make sure there are no spaces separating arguments
    for op_name, op in operations.items():
        if op_name == op_str:
            return op
        
    # if we're here, that should mean we have arguments to parse.
    for op_name, op in operations.items():
        if op_str.startswith(f"{op_name}("):
            break
    else: # if we're here, it means we never broke out of the `for` loop, i.e. couldn't find a matching op
        raise ValueError(f"{op_str} is not a supported operation. Supported operations: {[op_name for op in operations]}")
    
    arg_names = [p for p in signature(op).parameters]
    bracketed = ["{" + p + "}" for p in arg_names]
    template = f"{op_name}({','.join(bracketed)})"
    result = parse.parse(template, op_str)
    kargs = result.named
    kargs = {k:float(v) for k,v in kargs.items()} # coerce strings to floats
    return op(**kargs)
    

# signal_manipulations = 'bandpass(300, 500) | pow2 | pow2 | rms | pow2 | threshold(5e-6) ' # @param {'type':'string'}
signal_manipulations = 'harmonic | bandpass(300, 500) | novelty |  sustain(30) |  normalize | threshold(.25) ' # @param {'type':'string'}

manipulations = [m.strip() for m in signal_manipulations.split('|') if m.strip()]

# reset processing
driving_signal_massaged = driving_signal

# TODO: precompute time component, modify as needed.
#       - concretely, rms changes time domain from samples->frames.

show_spec=True
windowed_manipulations = ['rms', 'novelty']
manipulation_history = [] # this is necessary because certain manipulations shouldn't be repeated or otw can collide w each other
for idx, manipulation in enumerate(manipulations):
    print(manipulation)
    if manipulation in windowed_manipulations:
        if show_spec:
            show_spec=False
        else:
            warnings.warn(f"transform {manipulation} collides with a transform we've already performed. Skipping.")
            continue
    
    f = prepare_operation(manipulation)
    try:
        driving_signal_massaged = f(y=driving_signal_massaged, sr=sr)
    except (TypeError, librosa.ParameterError) as e: # this is sort of gross but fuck it
        print(e)
        f=f()
        driving_signal_massaged = f(driving_signal_massaged, sr)

    display_signal(driving_signal_massaged, sr, title=f"{idx}, {manipulation}", show_spec=show_spec)
else:
    # finalize signal
    driving_signal_massaged = np.abs(driving_signal_massaged).ravel()
    driving_signal_massaged /= max(driving_signal_massaged)
    display_signal(driving_signal_massaged, sr, title=f"Final Driving Signal", show_spec=False)


In [ ]:
# @title 2b. Run this cell when you're satisfied with the transformed signal

use_massaged_signal = True # @param {'type':'boolean'}
if use_massaged_signal:
    driving_signal = driving_signal_massaged


### Uncomment this snippet to write your driving signal to an audio file
#import soundfile
#soundfile.write('driving_signal.wav', driving_signal, sr)

normalized_signal = np.abs(driving_signal).ravel()
normalized_signal /= max(normalized_signal)

# TODO: manipulate the `frame_time` variable in synch with the signal. maybe track a "time_domain" variable ('frames'|'samples')

if show_spec:
    frame_time = librosa.samples_to_time(np.arange(len(normalized_signal)), sr=sr)
else:
    frame_time = librosa.frames_to_time(np.arange(len(normalized_signal)), sr=sr)

driving_signal_kf = kf.Curve({t:v for t,v in zip(frame_time, normalized_signal)})

plt.plot(frame_time, normalized_signal)
plt.xlabel('seconds')
plt.ylabel('intensity')
plt.title("Processed Audioreactivity Signal")
full_width_plot()

In [ ]:
# @title 3. Map the driving signal to parameter ranges 

# TODO: don't need this signal naming stuff. if anything, the signal we should be naming is the driving_signal

signal_name = 'noise_reactive' # @param {'type':'string'}
target_parameter = 'strength_curve' # @param {'type':'string'}
attr_hi= .08 # @param {'type':'number'}
attr_low = .02 # @param {'type':'number'}
inverse_relationship = False # @param {'type':'boolean'}

signals = [dict(
    signal_name=signal_name,
    attr_hi=attr_hi,
    attr_low=attr_low,
    inverse_relationship=inverse_relationship
)]

# signals =[
#     dict(
#         signal_name = 'noise_reactive',
#         attr_hi= .08,
#         attr_low = .02,
#         inverse_relationship = False,
#     ),
#     dict(
#         attr_hi=.8,
#         attr_low = .5,
#         inverse_relationship = True,
#         signal_name =  'audio_reactive_curve',
#     ),
#     dict(
#         attr_hi=30,
#         attr_low = 1,
#         inverse_relationship = False,
#         signal_name =  'dynamic_fps',
#     ),
# ]


# # 3b. Map the driving signal to parameter ranges
# reactive_signal_map = {
#     'audio_reactive_curve':'strength_curve',
#     'noise_reactive':'noise_add_curve',
#     'dynamic_fps':'dynamic_fps', # probably needs a better name...
# }

reactive_signal_map = {signal_name: target_parameter}

# TODO: make the above part more user friendly

# TODO: add the "applicable scenes" thing down here

##################################################################################################################
##################################################################################################################

# Hi user. You've gone to far. Don't worry about this stuff. Just the bit above the line.

for signal_params in signals:

    attr_hi= signal_params['attr_hi']
    attr_low = signal_params['attr_low']
    inverse_relationship = signal_params['inverse_relationship']
    signal_name = signal_params['signal_name']

    # TODO: instead of attaching the string, let's just build a curve object and use
    #       keyframed.Curve's native serialization. this was the whole point of doing it that way.

    #print(signal_params)

    for scene_idx, rec in enumerate(storyboard.prompt_starts):
        #print(scene_idx)
        if scene_idx == 0:
            prev_rec = rec
            continue
        start, end = prev_rec['start'], rec['start']

        # TODO: move this correction UP
        if scene_idx == 1:
          start = 0

        if prev_rec['frames'] < 1:
            continue
            
        curve_chunks = []        
        for frame_idx in range(prev_rec['frames']):
            curr_time = start + frame_idx * ifps
            signal_value = driving_signal_kf[curr_time]
            if inverse_relationship:
                signal_value = 1-signal_value
            attr_value = signal_value*(attr_hi-attr_low)+attr_low
            curve_chunks.append(f"{frame_idx}:({attr_value})")
        curve_str = ','.join(curve_chunks)
        prev_rec[signal_name] = curve_str
        prev_rec = rec
    else: # hate this.
        start, end = rec['start'], rec['end']
        if rec['frames'] > 0:
            curve_chunks = []
            for frame_idx in range(rec['frames']):
                curr_time = start + frame_idx * ifps
                signal_value = driving_signal_kf[curr_time]
                if inverse_relationship:
                    signal_value = 1-signal_value
                attr_value = signal_value*(attr_hi-attr_low)+attr_low
                curve_chunks.append(f"{frame_idx}:({attr_value})")
            curve_str = ','.join(curve_chunks)
            rec[signal_name] = curve_str

            
# TODO: write directly into rec.animation_args['arg']

# TODO: save keyframed objects for signals

if not storyboard.get('audioreactive'):
    storyboard.audioreactive = {'signals': [], 'reactive_signal_map': {}}

# TODO: check if these signals are already in the board
#       ...actually, will probably be simpler to just reshape this so each signal is a dict whose key is the name
storyboard.audioreactive.signals.append(signals) 

storyboard.audioreactive.reactive_signal_map.update(reactive_signal_map)
save_storyboard(storyboard)

show_storyboard(storyboard)

## $3.$ 🎬 Animate

In [ ]:
#####################################
# @title ## 🎨 Generate init images
#####################################

# @markdown If this cell throws errors, just run it again (with resume turned on)


# TODO: add archive/don't archive toggle...

prompt_starts = storyboard.prompt_starts
use_stability_api = workspace.use_stability_api
model_dir = workspace.model_dir

device = 'cuda'
model_id = "CompVis/stable-diffusion-v1-5"
download=True # TODO: pretty sure we shouldn't need to redownload here

model_dir = workspace.model_dir
model_path= str(Path(model_dir) / 'huggingface' / 'diffusers')

MAX_IM_PER_BATCH_HF = 1

#if 'get_image_for_prompt' not in locals():

if use_stability_api:
    import warnings
    from stability_sdk import client
    import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation


    def get_image_for_prompt(prompt, max_retries=5, **kwargs):
        return get_image_for_prompt_sai(prompt, max_retries=5, **kwargs)


    # leverage stability API internal parallelism for batch variation requests
    def get_variations_w_init(prompt, init_image, n_variations=2, image_consistency=.7, **kwargs):
          return list(
              get_image_for_prompt(
                  prompt=prompt, 
                  init_image=init_image, 
                  start_schedule=(1-image_consistency), 
                  #num_samples=n_variations,
                  samples=n_variations,
                  **kwargs,
              )
          )
                    
else:

    if download:
        img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_id,
            revision="fp16", 
            torch_dtype=torch.float16,
            use_auth_token=True
        )
        img2img = img2img.to(device)
        img2img.save_pretrained(model_path)
    else:
        img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_path,
            local_files_only=True
        ).to(device)

    text2img = StableDiffusionPipeline(
        vae=img2img.vae,
        text_encoder=img2img.text_encoder,
        tokenizer=img2img.tokenizer,
        unet=img2img.unet,
        feature_extractor=img2img.feature_extractor,
        scheduler=img2img.scheduler,
        safety_checker=img2img.safety_checker,
    )
    text2img.enable_attention_slicing()
    img2img.enable_attention_slicing()


    def get_image_for_prompt_hf(
        prompt,
        **kwargs
    ):
        if 'init_image' in kwargs:
            kwargs['image'] = kwargs.pop('init_image')
        if 'start_schedule' in kwargs:
            kwargs['strength'] = kwargs.pop('start_schedule')
        if 'image_consistency' in kwargs:
            kwargs['strength'] = 1-kwargs.pop('image_consistency')
        f = text2img if kwargs.get('image') is None else img2img
        n_retries = 5
        with autocast(device):
            return f(prompt, **kwargs)
            # while n_retries > 0:
            #     n_retries-=1
            #     result = f(prompt, **kwargs)
            #     if not any(result.nsfw_content_detected):
            #         return result.images
            #     else:
            #         print(f"nsfw content detectected. retries remaining: {n_retries}")

    def get_image_for_prompt(*args, **kwargs):
        # if 'init_image' in kwargs:
        #     kwargs['image'] = kwargs.pop('init_image')
        # if 'start_schedule' in kwargs:
        #     kwargs['strength'] = kwargs.pop('start_schedule')
        # if 'image_consistency' in kwargs:
        #     kwargs['strength'] = 1-kwargs.pop('image_consistency')
        n_retries = 5
        while n_retries > 0:
                n_retries-=1
                result = get_image_for_prompt_hf(*args, **kwargs)
                if not any(result.nsfw_content_detected):
                    return result.images
                else:
                    print(f"nsfw content detectected. retries remaining: {n_retries}")


    # TODO: (HF) request multiple images in single request
    def get_variations_w_init(prompt, init_image, **kwargs):
        #if 'n_variations' in kwargs:
        #    kwargs['num_images_per_prompt'] = min(MAX_IM_PER_BATCH_HF, kwargs.pop('n_variations'))
        n_variations = kwargs.pop('n_variations')
        while n_variations > 0:
            n_imgs_this_batch = min(MAX_IM_PER_BATCH_HF, n_variations)
            #for response in get_image_for_prompt(
            response = get_image_for_prompt_hf(
                prompt=prompt, 
                init_image=init_image, 
                num_images_per_prompt=n_imgs_this_batch,
                return_dict=True,
                **kwargs,
            )
            print(response)
            for img, nsfw in zip(response['images'], response['nsfw_content_detected']):
                if not nsfw:
                    yield img
                    n_variations -= 1
                else:
                    warnings.warn("NSFW classifier triggered. Trying again.")



##################
##  PARAMETERS  ##
##################

d_ = dict(
    _=''
    , height = 512 # @param {type:'integer'}
    , width = 512 # @param {type:'integer'}
    # TODO: pretty sure can delete this
    #, display_frames_as_we_get_them = True # @param {type:'boolean'}
)
d_.pop('_')

#regenerate_all_init_images = True # @param {type:'boolean'}
regenerate_all_init_images = False # @param {type:'boolean'}

# TODO: make this an integer
prompt_lag = True # @param {type:'boolean'}

# @markdown `prompt_lag` - Extend prompt with lyrics from previous frame. Can improve temporal consistency of narrative. 
# @markdown  Especially useful for lyrics segmented into short prompts.

#TODO: check from storyboard

# regenerate all images if the theme prompt has changed or user specifies
#if d_['theme_prompt'] != storyboard.params.get('theme_prompt'):
#    regenerate_all_init_images = True


storyboard.params.update(d_)

if regenerate_all_init_images:
    for i, rec in enumerate(prompt_starts):
        rec['frame0_fpath'] = None
        archive_images(i, root=root)
    print("archival process complete")

# anchor images will be regenerated if there's no associated frame0_fpath
# regenerate specific images if
# * manually tagged by user in df_regen
# * associated fpath doesn't exist (i.e. deleted)
# TODO: dump df_regen stuff
# TODO: move archival stuff...
if 'df_regen' in locals():
    for i, _ in df_regen.iterrows():
        rec = prompt_starts[i]
        regen = not _['keep']
        # need to check this elsewhere
        if rec.get('frame0_fpath') is None:
            regen = True
        elif not Path(rec['frame0_fpath']).exists():
            regen=True
        if regen:
            rec['frame0_fpath'] = None
            print(rec)
            archive_images(i, root=root)
    print("archival process complete")


theme_prompts = storyboard.params.theme_prompt
height = storyboard.params.height
width = storyboard.params.width

proj_name = workspace.active_project


## Main loop ##

print("Ensuring each prompt has an associated image")
for idx, rec in enumerate(prompt_starts):
    #print(idx, rec)
    theme = rec.get('_theme')
    prompt = rec.get('prompt')
    if not prompt:
        prompt = f"{rec['text']}, {theme}"
    
        if prompt_lag and (idx > 0):
            rec_prev = prompt_starts[idx -1]
            prev_text = rec_prev.get('text')
            if not prev_text:
                prev_text = rec_prev.get('prompt').split(',')[0]
            this_text = rec.get('text')
            if this_text:
                prompt = f"{prev_text} {this_text}, {theme}"
            else:
                prompt = rec_prev['_prompt']
    rec['_prompt'] = prompt
    
    print(
        f"scene: {idx}\t time: {rec['start']}\n"
        f"spoken text: {rec.get('text')}\n"
        f"image prompt: {rec['_prompt']}\n"
    )
    need_image = True
    if rec.get('frame0_fpath') is not None:
        if Path(rec['frame0_fpath']).exists():
            need_image = False
    if need_image:
        
        # TODO: get generation settings from rec.animation_args ## needs a more general name
        init_image = list(get_image_for_prompt(
              rec['_prompt'],
              height=height,
              width=width,
              )
          )[0]
        
        # TODO: save_frame doesn't need to be a function.
        rec['frame0_fpath'] = save_frame(
            init_image,
            idx,
            root_path = root / 'frames',
            name='anchor',
            )

        print(rec.get('text'))
        display(init_image)

# TODO: regen picks up at first frame that needs it correctly, 
#       but then continues on overwriting all subsequent frames.
#       Fix it to regain ability to delete specific init images to have just those regened

##############
# checkpoint #
##############

# TODO: recognize previous generations

storyboard.prompt_starts = prompt_starts

save_storyboard(storyboard)

In [ ]:
# @markdown Did one of your prompts trigger a safety filter? modify it here.

scene_id = 0 # @param {type:'integer'}
new_caption = "" # @param {type:'string'}

###########################################################

if new_caption:
    old_caption = storyboard.prompt_starts[scene_id].text
    storyboard.prompt_starts[scene_id].text = new_caption
    print(f"{old_caption} -> {new_caption}")

In [ ]:
# @title ## 🚀 Animate!

# Fuck it, this should be consistent anyway, right?
# -------------------------------------------------
# load storyboard from disk before animating if you 
# made changes that you want to be respected
workspace, storyboard = load_storyboard()

# @markdown Negative prompt currently only used with animation mode img2img

#TODO: mechanism to archive/delete just the animation frames

# TODO: uh... you don't have to go home, but you can't stay here.
negative_prompt = 'blurry, low detail, low quality, unfinished' # @param {type:'string'}
negative_prompt_weight = -1 # @param {type:'number'}


# ... why is this so hard. TODO: make this whole cell simpler.


from keyframed import SmoothCurve


# TODO: write a (different name though) scenes.txt for each scene, so we can get fancier with how we animate and generate

print("Animating Sequence")
for idx, rec in enumerate(storyboard.prompt_starts):
    
    # at least it'll be consistent I guess...
    if rec.get('frame0_fpath') is None:
        print(f"skipping scene {idx}, no init image detected")
    
    print(f"Animating scene {idx}")
    
    
    # default mode contingent on api mode
    animation_mode = rec.get('animation_mode', 'default').lower()
    if animation_mode == 'default':
        animation_mode = 'img2img' if workspace.use_stability_api else 'variations tsp'
    print(f"animation mode: {animation_mode}")
        
    # figure out how many images we've already generated
    #TODO: "root" should be outpath, not project_root. 
    #      push up any "/frames" logic in get_image_sequence
    images_fpaths = get_image_sequence(idx, root=workspace.project_root)
    images_fpaths = sorted(images_fpaths, key=os.path.getmtime)
    
    curr_variation_count = len(images_fpaths)
    print(f"curr_variation_count:{curr_variation_count}")

    ##########################################################################
    
    ### calculate total number of images needed per scene depends on animation mode
    ### and figure out init_image/init_image_fpath
    
    # `animation_mode=None`
    tot_variations = 0
    
    if any(adj in animation_mode 
           for adj in ('jittered','variations')):
        
        init_image_path = images_fpaths[0]
        init_image = Image.open(init_image_path)
        
        # TODO: generalize this, probably means pushing tot_variations into some other settings object
        # TODO: create some sort of get_default('param_name') or get overrides that wraps the procedure of checking for a value in the rec or the sotryboard.params
        # TODO: rename the xx_variations" vars. `tot_variations` = total number of images that need to be generated for this scene (function of scene and animation mode)
        
        # permits user to specify more variations for a specific scene
        #tot_variations = rec.get('n_variations', storyboard.params.n_variations)
        tot_variations = rec.get('n_variations', storyboard.params.get('n_variations',5))
        tot_variations = min(tot_variations, rec['frames'])

        image_consistency = rec.get('image_consistency', storyboard.params.get('image_consistency', .75)) # 1-Curve(args.noise) or whatever
        
    if 'img2img' in animation_mode:
        
        init_image_path = images_fpaths[-1]
        
        tot_variations = rec['frames']
        print(f"tot_variations:{tot_variations}")
        
        args = AnimationArgs(**rec.get('animation_args',{}))
    
        
        ### inject tweaks to animation (maybe this could be a callback/hook or something?)

        # TODO: expose this as an option or something
#         sign = (-1)**(idx%2)
#         value = sign * 0.1
#         args.translation_x = f"0:({value})"
#         args.rotation_y = f"0:({-value})"

    ##########################################################################   

    tot_variations -= curr_variation_count  # only generate variations we still need
    
    if tot_variations < 1:
        print("No animation frames required for this scene. Checking next scene")
        continue

    print(f"tot_variations to request:{tot_variations}")

    prompt = rec['_prompt']
    
    ##########################################################################
    
    if 'variations' in animation_mode:
        
        image_variations = get_variations_w_init(
            prompt = prompt, 
            init_image = init_image, 
            image_consistency = image_consistency, # TODO: align with animation args
            n_variations = tot_variations,
        )
        for img in image_variations:
            save_frame(
                img,
                idx,
                root_path= root / 'frames',
            )
            display(img)
        
    elif 'img2img' in animation_mode:
        from omegaconf.errors import ConfigAttributeError
        
        args.init_image = init_image_path
        args.max_frames = tot_variations
        try:
            args.fps = storyboard.params.fps # this shit is gonna be a mess...
        except ConfigAttributeError:
            pass
        
        # TODO: make this more flexible (e.g. pull everything from animation_args...)
        animation_prompts = {0:prompt}
        
        # TODO: this shouldn't be specific to SAI animation
        #       `args` object for other anim modes as well, then wrap those modes in functions?
        
        # TODO: prove out generalized-ness with curved fps
        
        for k,v in storyboard.audioreactive.reactive_signal_map.items():
            setattr(args, v, rec[k])

        animator = Animator(
            api_context=context,
            animation_prompts=animation_prompts,
            args=args,
            out_dir=None, #out_dir,    
            negative_prompt=negative_prompt,
            negative_prompt_weight=negative_prompt_weight,
            #resume=len(resume_timestring) != 0
            resume=False,
        )

########################################################
    
        # TODO: push this out of the loop? I feel like what I want to be able to do
        #       is loop over each frame in the scene, within the rec loop, in such a way that we go back up to 
        #       the top of the loop between images. At which point, it's like... sheesh.
        #       basically what I want to be able to do is modify settings between frames, 
        #       since we're constrained by the data types of the AnimationArgs params
        
        for img in tqdm(animator.render(), initial=animator.start_frame_idx, total=args.max_frames):
            # TODO: this isn't super reliable, often crashes colab
            from IPython.display import clear_output
            clear_output(wait=True)
            display(img)
            new_frame_fpath = save_frame(
                img, idx, root_path=root/'frames',
                name=f"{tot_variations}" # it'll be reversed, but it's not nothing
            )
            tot_variations-=1 # for naming, which needs to change anyway


        # TODO: async this, as well as the image saving
        # TODO: append entry to a scenes.txt for the scene here. This gets us around cluttering the storyboard with unneeded file paths.
        #       ...I should probably stop doing everything in text files and just bite the bullet and use a database. export a text file 
        #       for the user which can be loaded, but persist the granular information I want here in relational tables.
        
    else:
        raise NotImplementedError
        
# TODO: parallelize generations across scenes. 
#       i.e. maybe it's non trivial to parallelize generations within a scene,
#       but we can do several scenes in parallel. need more control over how
#       the animator constructs/emits requests. want it to construct requests
#       for several scenes, then multiplex the request objects into a single
#       larger request, which would then need to have responses demuxed

# TODO: [keyframed ] max-binning/aggregation (sync) functionality
#       ... or is this something it already supports?
# TODO: [keyframed] lag aggregation
# TODO: [keyframed] "simplify" operation
# TODO: [keyframed] ability to specify operations relative to keyframe indices, e.g. interpolate from V to 0 over K frames for each keyframe


In [ ]:
# @title ## 📺 Compile your video and enjoy your animation!

# TODO: print output path

# TODO: skip tsp if n_variations ==1

# TODO: change frame-write names so this will be compatible with an ffmpeg one-liner to generate preview animations
#       - mixed feelings here. 

# TODO: ffmpeg script conditional on animation mode

########################
# rendering parameters #
########################

# @markdown `add_caption` - Whether or not to overlay the prompt text on the image

# @markdown  `upscale`: Naively (lanczos interpolation) upscale video 2x. This can be a way to force
# @markdown  services like youtube to deliver your video without mangling it with compression
# @markdown  artifacts. Thanks [@gandamu_ml](https://twitter.com/gandamu_ml) for this trick!

output_filename = 'output.mp4' # @param {type:'string'}
add_caption = False # @param {type:'boolean'}
upscale = False # @param {type:'boolean'}
download_video = True # @param {type:'boolean'}

# @markdown NB: Your video will probably download way faster from https://drive.google.com

#########################

final_output_filename = str( root / output_filename )
storyboard.params.output_filename = final_output_filename


#fps = storyboard.params.fps # TODO: change ffmpeg command so we don't need this


#####################################

from keyframed import Curve # sheesh...

# prep everything...
ffmpeg_cmd_script = ""
for idx, rec in enumerate(storyboard.prompt_starts):
    scene_fps = Curve(storyboard.params.fps)
    if rec.get('dynamic_fps'): # should probably just call this fps...
        scene_fps = curve_from_cn_string(rec['dynamic_fps'])
        
    im_paths = get_image_sequence(idx, root)
    if rec.get('animation_mode') == 'variations tsp':
        if rec.get('frame_order'):
            im_paths = rec['frame_order']
        else:
            print(f"computing frame order for scene {idx}")
            images = [Image.open(fp) for fp in im_paths]
            try:
                frame_order = tsp_sort(images)
                im_paths = [im_paths[j] for j in frame_order]
                images = [images[j] for j in frame_order]
            except ValueError:
                pass

            # TODO: persist frame order to storyboard for variations_tsp. or not? mixed feelings again.
            rec['frame_order'] = im_paths
            #save_storyboard(storyboard) # make sure we aren't doing this frame_order nonsense for fancy animations
            
    elif rec.get('animation_mode') == 'img2img':
        # this is sort of a hack but it works for now
        im_paths = sorted(im_paths, key=os.path.getmtime)

    images = [Image.open(fp) for fp in im_paths]

    if add_caption:
        new_paths = []
        #images_captioned = [add_caption2image(im, rec['prompt']) for im in images]
        #images_captioned = [add_caption2image(im, rec['text']) for im in images]
        #for fp, im in zip(im_paths, images_captioned):
        for fp, im in zip(im_paths, images):
            fp = Path(fp)
            #fp = fp.with_stem(fp.stem + '-captioned')
            fp = fp.parent / 'captioned' / fp.name
            fp.parent.mkdir(exist_ok=True, parents=True)
            if not rec.get('inferred_subscene', False):
                im = add_caption2image(im, rec['text']) # TODO: separate "caption" attribute on the rec
            im.save(fp)
            new_paths.append(fp)
        im_paths = new_paths
    
    try:
        frame_picker = cycle(im_paths)
        for frame_idx in range(rec.frames):
            fpath = Path(next(frame_picker))
            fps = scene_fps[frame_idx] # ain't I fancy # ... this was a good idea but requires that I compute things differently. deal with it later.
            ffmpeg_cmd_script += f"file '{fpath.absolute()}'\nduration {1/fps}\n"

        with open(root/'scenes.txt', 'w') as f:
            f.write(ffmpeg_cmd_script)
    except:
        print("maybe missing some frames? your video might be incomplete, sorry. try the resuming procedure.")
        print()
        break

if upscale:
    height=storyboard.params.height
    width=storyboard.params.width
    !ffmpeg -y -f concat -safe 0 -i {root/'scenes.txt'} -i "{storyboard.params.audio_fpath}" -r {storyboard.params.fps} -pix_fmt yuv420p -crf 25 -preset veryslow -vf scale={2*width}x{2*height}:flags=lanczos -shortest {storyboard.params.output_filename}
else:
    !ffmpeg -y -f concat -safe 0 -i {root/'scenes.txt'} -i "{storyboard.params.audio_fpath}" -r {storyboard.params.fps} -pix_fmt yuv420p -crf 25 -preset veryfast -shortest {storyboard.params.output_filename}


# EASTER EGG FEATURE
#  NB: only embed short videos
embed_video_in_notebook = False

output_filename = storyboard.params.output_filename

if download_video and not local:
    from google.colab import files
    files.download(output_filename)

if embed_video_in_notebook:
    from IPython.display import display, Video
    display(Video(output_filename, embed=True))

#!ffmpeg -y -f concat -safe 0 -i {root/'scenes.txt'} -vn -i "{video_assets_meta_record['video_fpath']}" -r {storyboard.params.fps} -pix_fmt yuv420p -crf 25 -preset veryfast -shortest test_alt_audio.mp4

# ⚖️ I put on my robe and lawyer hat

### Notebook license

This notebook and the accompanying [git repository](https://github.com/dmarx/video-killed-the-radio-star/) and its contents are shared under the MIT license.

<!-- Note to self: lawyers should really be forced to use some sort of markup or pseudocode to eliminate ambiguity 

...oh shit, if laws were actually described in code, we could just run queries against it
-->

```
MIT License

Copyright (c) 2022 David Marx

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```

### DreamStudio API TOS

The default behavior of this notebook uses the [DreamStudio](https://beta.dreamstudio.ai/) API to generate images. Users of the DreamStudio API are subject to the DreamStudio usage terms: https://beta.dreamstudio.ai/terms-of-service

### Stable Diffusion

As of the date of this writing (2022-09-29), all publicly available model checkpoints are subject to the restrictions of the Open RAIL license: https://huggingface.co/spaces/CompVis/stable-diffusion-license. 

